# Pruebas Filtro:

- Lenguaje Filtro (Python)
- Crear sistema de web scraping, y crear dataset de al menos 5 categorías, con datos obtenidos a       través del web scraping.
- Las categorías deben de estar numéricamente balanceadas en cantidad.
- Se debe crear un procedimiento automatizado de preprocesamiento .
- Se debe desarrollar un modelo de redes neuronales artificiales con tensor flow/keras , para el       procesamiento de este dataset.
- Se debe crear una api rest , que proporcione por medio de métodos http , un ingreso de datos y una   salida de la clasificación del mismo.

#### Actualizamos el inslalador pip e instalamos las biblotecas para hacer el scraping y acceder a la página a scrapear

In [ ]:
# ! pip install --upgrade pip
# ! pip install beautifulsoup4
# ! pip install requests

## Definimos las 5 categorías

## Categorías

1. Edad antigua
2. Edad media
3. Edad moderna
4. Edad contemporánea
5. Prehistoria

# Clases para periodismo

- Política y economía
- Ciencia, tecnología, salud, cultura, innovación y emprendedorismo
- Viajes, turismo
- Espectáculos, recreación, gastronomía y sociales
- Deportes

In [2]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

In [ ]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [ ]:
clases = 5

## Paso 1 - Scraping de textos crudos

In [4]:
webs = pd.ExcelFile("Webs.xls")

In [5]:
print(webs.sheet_names)

['Categorias']


In [10]:
df = webs.parse(webs.sheet_names[0])
clases_df = set(df["CÓDIGO"].value_counts().index)
df

,URL,NOMBRE ARCHIVO,CLASE,CÓDIGO
0,https://en.wikipedia.org/wiki/Alexander_the_Great,Alexander_the_Great,EDAD ANTIGUA,1
1,https://en.wikipedia.org/wiki/History_of_scien...,History_of_science_in_classical_antiquity,EDAD ANTIGUA,1
2,https://en.wikipedia.org/wiki/Roman_technology,Roman_technology,EDAD ANTIGUA,1
3,https://en.wikipedia.org/wiki/Boudica,Boudica,EDAD ANTIGUA,1
4,https://en.wikipedia.org/wiki/Julius_Caesar,Julius_Caesar,EDAD ANTIGUA,1
...,...,...,...,...
84,https://en.wikipedia.org/wiki/Prehistory_of_Au...,Prehistory_of_Australia,PREHISTORIA,5
85,https://en.wikipedia.org/wiki/Control_of_fire_...,Control_of_fire_by_early_humans,PREHISTORIA,5
86,https://en.wikipedia.org/wiki/Archaeology_of_t...,Archaeology_of_the_Americas,PREHISTORIA,5
87,https://en.wikipedia.org/wiki/Indigenous_Austr...,Indigenous_Australian_art,PREHISTORIA,5


In [7]:
textos = pd.DataFrame(columns=['Texto', 'Clase'])
for i in df.index: 
    print("i", i)
    url = df["URL"][i]
    req = requests.get(url)
    statusCode = req.status_code
    html = BeautifulSoup(req.text, "html.parser")
    if statusCode == 200:
        paragraphs = html.select("p")
        cont = 0
        for para in paragraphs:
            cont += 1 
            parrafo = ""
            print("parrafo",parrafo) 
            print("i",i) 
            print("df['CÓDIGO'][i]",df["CÓDIGO"][i]) 
            print('df["NOMBRE ARCHIVO"][i]',df["NOMBRE ARCHIVO"][i])
            print("cont",cont) 
            guardarComo = str(df["CÓDIGO"][i]) + "/"  + df["NOMBRE ARCHIVO"][i] + str(cont)
            print("guardarComo",guardarComo)  
            parrafo = open(guardarComo,"w",encoding = "utf-8") 
            parrafo.write(para.text)
            textos = textos.append({'Texto': para.text, 'Clase':df["CÓDIGO"][i]}, ignore_index=True)
            parrafo.close() 

AttributeError: 'dict' object has no attribute 'index'

In [ ]:
textos.sample(10)

In [ ]:
textos["Texto"][6177]

In [ ]:
textos.to_csv("H01 - HistoriaTextosCrudos.csv")

## Paso 2 - Preprocesamiento de textos

In [ ]:
textosCrudos = pd.read_csv("H01 - HistoriaTextosCrudos.csv")
textosCrudos = textosCrudos.iloc[:,1:]
textosCrudos.head(60)

In [ ]:
textosCrudos.groupby(by=["Clase"]).count()

In [ ]:
textosCrudos["TextoProcesado"] = None

In [ ]:
textosCrudos.sample(10)

In [ ]:
cont = 0
stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()

textos      = pd.DataFrame(columns = [ 'Texto' , 'Clase'])

for i in textosCrudos.index:
    cont += 1
    if cont > 10:
        break
    print()    
    texto_tokens = tokenizer.tokenize(textosCrudos["Texto"][i])
    texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
    texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
    for t in range(len(texto_stem)//20):
        j = 20 * t
        palabras  = texto_stem[ j : 20 * (t+1) ] 
        textos   = textos.append({'Texto' : palabras, 'Clase' : textosCrudos["Clase"][i] }, ignore_index = True)
    
    print(textosCrudos["Texto"][i])
    print(texto_tokens)
    print() 
    print (texto_clean)
    print() 
    print (texto_stem)
textos

In [ ]:
for i in textos.index:
    print()
    print(textos["Texto"][i])
textos    

In [ ]:
stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()

textos      = pd.DataFrame(columns = [ 'Texto' , 'Clase'])

for i in textosCrudos.index: 
    texto_tokens = tokenizer.tokenize(textosCrudos["Texto"][i])
    texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
    texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
    for t in range(len(texto_stem) // 20):
        j = 20 * t
        palabras  = texto_stem[ j : 20 * (t+1) ] 
        textos   = textos.append({'Texto' : palabras, 'Clase' : textosCrudos["Clase"][i] }, ignore_index = True)
        clear_output( wait = True )
        print(f"Creando Textos {i} de {len(textosCrudos.index)}")
textos

In [ ]:
cont = 0
for i in textos.index:
    cont += 1
    if cont > 10:
        break
    print()    
    print("longitud:",len(textos["Texto"][i].split()), textos["Texto"][i])    

In [ ]:
textos.to_csv("H02 - Historia.csv")

## Vocabulario y frecuencias

In [ ]:
textos = pd.read_csv("H02 - Historia.csv")
textos = textos.iloc[:,1:]
textos.sample(10)

In [ ]:
textos.groupby(by=["Clase"]).count()

In [ ]:
cont = 58
for m in textos.index:
    if m < cont:
        continue
    cont +=1
    if cont > 61:
        break
    print()    
    print(m)    
    print(type(textos.loc[m].Texto))        
    print(textos.loc[m].Texto)    
    print()
    print(textos.loc[m].Texto.split(","))  
    lista = textos.loc[m].Texto.split(",")
    cont2 = 0
    print()
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]     
        print()
        print(len(word),word,word[-1:],len(word[-1:]))
        if word[-1:] == "'":
            print("True")
            word = word[:-1]
            print("queda:",word)
        else:
            print("False")
            

In [ ]:
vocabulario = set() 
for m in textos.index:
    lista = textos.loc[m].Texto.split(",")
    cont2 = 0
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]   
        if word[-1:] == "'":
            word = word[:-1]
        vocabulario.add(word)
        clear_output( wait = True )
        print(f"Creando Vocabulario {m} de {len(textos.index)}")

In [ ]:
len(vocabulario)

In [ ]:
"hunter-gather" in vocabulario

In [ ]:
save_obj(vocabulario, "Vocabulario")

In [ ]:
for word in vocabulario:
    print(word)    

In [ ]:
vocabulario = load_obj("Vocabulario")

In [ ]:
frecuencias = defaultdict(int)
cont = 0 
for i in textos.index:
    cont += 1
    print()
    print("Contador",cont)
    if cont > 10:
        break
    lista = textos.loc[m].Texto.split(",")
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]   
        if word[-1:] == "'":
            word = word[:-1]
        print(word)
        # word = "pirulito"
        assert (word in vocabulario),"La palabra no está en el vocabulario!"
        frecuencias[(word, textos.loc[i].Clase)] += 1
        # clear_output( wait = True )
        # print(f"Creando Frecuencias {i} de {len(textos)}")

In [ ]:
frecuencias = defaultdict(int)
for i in textos.index:
    lista = textos.loc[m].Texto.split(",")
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]   
        if word[-1:] == "'":
            word = word[:-1]
        print(word)
        # word = "pirulito"
        assert (word in vocabulario),"La palabra no está en el vocabulario!"
        frecuencias[(word, textos.loc[i].Clase)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

In [ ]:
save_obj(frecuencias, "Frecuencias")

In [ ]:
frecuencias = load_obj("Frecuencias")

In [ ]:
len(frecuencias)

In [ ]:
frecuencias

In [ ]:
def limpiarWord(word):
    word = word[:-1]
    return word

In [ ]:
# create frequency dictionary
freqs = {}
cont = 0
for i in textos.index:  
    cont += 1
    if cont > 100000:
        break
    print()    
    print(i)    
    words = textos.loc[i,'Texto'].split(",")
    clase = textos.loc[i,'Clase']
    cont2 = 0
    for word in words:
        cont2 += 1
        if cont2 == len(words):
            word = word[:-1] 
        word = word[2:-1]   

      
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        
        print(word)
        # word = "pirulito"
        assert (word in vocabulario),"La palabra " + word + "no está en el vocabulario!"
        pair = (word, clase)
        freqs[pair] = freqs.get(pair, clase) + 1
# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')  

In [ ]:
print(textos["Texto"][60])

In [ ]:
print(textosCrudos["Texto"][30])

In [ ]:
# calculate V, the number of unique words in the vocabulary
vocab = set([pair[0] for pair in freqs.keys()])
V = len(vocab)
print(V)

In [ ]:
save_obj(freqs, "Frecuencias")

In [ ]:
frecuencias = load_obj("Frecuencias")

In [ ]:
len(frecuencias)

In [ ]:
def extract_features(words, freqs, vocabulario):
    '''
    Input: 
        words: a list of words for one input
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,6)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = words
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 6)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    # loop through each word in the list of words
    cont2 = 0
    for word in word_l:
        cont2 += 1
        if cont2 == len(words):
            word = word[:-1] 
        word = word[2:-1]   
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        print(word)
        assert (word in vocabulario),"La palabra " + word + "no está en el vocabulario!"

        # increment the word count for the positive label 1
        for i in range(1,6):
            x[0,i] += freqs.get((word, i),0)        
    assert(x.shape == (1, 6))
    return x

In [ ]:
textosFreq = textos.copy()
for j in range(1, nclases):
    textosFreq["Clase" + str(j)] = None
textosFreq    

In [ ]:
for i in textos.index:
    words = textos.loc[:,"Texto"][i].split(",")    
    clases = extract_features(words, freqs, vocabulario)
    nclases = len(clases[0]) 
    for j in range(1, nclases):
        textosFreq["Clase" + str(j)][i] = clases[0,j]
textosFreq        

In [ ]:
textosFreq.sample(10)

In [ ]:
textosFreq.to_csv("H03 - HistoriaFreq.csv")

In [ ]:
frecuenciasMaximas = {} 
for word in vocabulario:    
    freqWord = []
    print(word)
    for k in range(clases + 1):
        freqWord.append(0)   
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)    
    for k in range(1, len(freqWord)):
        frecuenciasMaximas[word] = np.argmax(freqWord)   
clasesPalabrasFM = {}
for k in range(1,clases + 1):
    clasesPalabrasFM[k] = []     
for word in frecuenciasMaximas.keys():
    clasesPalabrasFM[frecuenciasMaximas[word]].append(word)
totV = 0        

for k in range(1, clases + 1):
    print()
    print("Clase:",k)
    print(len(clasesPalabrasFM[k]))
    totV += len(clasesPalabrasFM[k])  
assert(totV == len(vocabulario)),"PALABRAS FALTANTES"    
    
frecuenciasMaxDetalle = {}     
    
for word, value in frecuenciasMaximas.items():
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)     
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)        
    
    if sum(freqWord) == frecuencias[word, np.argmax(freqWord)]:
        # print("SOLO APARECE EN UNA CLASE")
        rate = 99
    else:    
        rate = int(100 * frecuencias[word, np.argmax(freqWord)] / sum(freqWord))
    
    frecuenciasMaxDetalle[word] = value, rate - 19
    
print()
# print(frecuenciasMaxDetalle)


print()
print(len(frecuenciasMaxDetalle))
for i in frecuenciasMaxDetalle.items():
    print(i)



for word in vocabulario:
    print()
    print(word)
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)   
    for k in range(1, len(freqWord)):   
        freqWord[k] = frecuencias.get((word, k), 0)    
    print(freqWord)        
    print(frecuenciasMaxDetalle[word])
    print("Clase a ponderar",np.argmax(freqWord))
    print("Valor a ponderar",freqWord[np.argmax(freqWord)])
    print("Factor de ponderacion",frecuenciasMaxDetalle[word][1])
    print("Valor a Ponderar", frecuencias[(word, np.argmax(freqWord))]  )
    print("Factor de ponderacion", (frecuenciasMaxDetalle[word][1] + 100) / 100) 
    print("Valor Pond.",(int (frecuencias[(word, np.argmax(freqWord))]  * (frecuenciasMaxDetalle[word][1] + 100) / 100)))
  

In [ ]:
vocabulario

In [ ]:
frecuencias[('panina', 5)]

In [ ]:
cont = 0
for key in frecuenciasMaximas.keys():
    cont += 1
    if cont > 10:
        break
    print()
    print(key, frecuenciasMaximas[key])
    
